In [211]:
import os
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## Step 1: Data Preparation

In [212]:
species_mapping = pd.read_csv("traps_main_species_twd97.csv")

data_dir = 'D:/Yehmh/test_py/202301/P00074_transect_1/1m_1m/known'
Folder_names = "T1S"
image_size = 34

X = []  # Features
y = []  # Labels

for folder in os.listdir(data_dir):
    if folder.startswith(Folder_names):
        species_info = species_mapping.loc[species_mapping["Plot"] == folder]
        if not species_info.empty and not pd.isna(species_info["SpeciesID"].values[0]):
            species = species_info["SpeciesID"].values[0]
            for file in os.listdir(os.path.join(data_dir, folder)):
                if file.endswith(".tif"):
                    image_path = os.path.join(data_dir, folder, file)
                    image = cv2.imread(image_path)                
                    # image = cv2.resize(image, (image_size, image_size))  # Resize image if necessary
                    # image = cv2.resize(image, (64, 64))  # Resize image if necessary
                    X.append(image)
                    y.append(species)
        else:
            print(f"Folder {folder} does not have a corresponding species label.")

Folder T1S1 does not have a corresponding species label.


Folder T1S2 does not have a corresponding species label.


## Step 2: Data Preprocessing

In [213]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [214]:
print(label_encoder.classes_)
print(y)

[1. 2. 5. 7.]
[3 3 3 ... 2 2 2]


## Step 3: Define transformations and create datasets

In [215]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize image data
])

train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

## Step 4: Create data loaders

In [216]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# test_loader is created with shuffle=False. 
# Ensures that the evaluation process remains consistent across different evaluations 
# and that the model is tested on the same data distribution every time

## Step 5: Define the CNN model

In [217]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        # Define the convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)     # (in_channels (rgb), out_channels, filter_num, stride, padding), default stride = 1
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        # Define the fully connected layers
        # self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Define the forward pass through the network
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = nn.functional.max_pool2d(x, 2)
        # x = x.view(-1, 64 * 16 * 16)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Step 6: Initialize model. loss function, and optimizer

In [218]:
model = CNN(num_classes=len(label_encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [219]:
print(len(label_encoder.classes_))

4


## Step 7: Train the model

In [220]:
def accuracy(predictions, laels):
    pred = torch.max(prediction.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [221]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        # print("Batch Shape - Images:", images.shape)
        # print("Batch Shape - Labels:", labels.shape)

        optimizer.zero_grad()   # Zero the gradient
        outputs = model(images) # Forward pass: computer predicted outputs
        # print(outputs.shape)
        loss = criterion(outputs, labels) # Compute the loss
        loss.backward()     # Backward pass: compute gradient of the loss with respect to model parameters
        optimizer.step()    # Update the model parameters based on the gradients

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

Epoch [1/10], Loss: 1.1590547561645508
Epoch [2/10], Loss: 1.0803948640823364
Epoch [3/10], Loss: 0.8649986982345581
Epoch [4/10], Loss: 1.1307172775268555
Epoch [5/10], Loss: 0.8650143146514893
Epoch [6/10], Loss: 1.1389328241348267
Epoch [7/10], Loss: 0.7979801893234253
Epoch [8/10], Loss: 0.8825727701187134
Epoch [9/10], Loss: 0.5664574503898621
Epoch [10/10], Loss: 1.3208736181259155


## Step 8: Evaluate the model

In [222]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Test Accuracy: {:.2f}%'.format(100 * accuracy))

Test Accuracy: 64.27%
